#  GPT-2 Generation and Fine-Tuning

This is adjusted for use in Reid Brockmeier's (rbrockmeier2@unl.edu) generative-text project. Most of the unused segments were removed, and other changes were mostly adjustments of values.

This notebook explores GPT-2 (Generative Pretrained Transformer-2) from OpenAI. Read more about it [here](https://openai.com/blog/better-language-models/).

Activities include:

0. Setup
1. Generate samples from pre-trained gpt-3 model
2. Fine-tune gpt-2 on text of your choosing. 

Adapted by Robert Twomey (rtwomey@unl.edu) for Machine Learning for the Arts SP22 from this [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) by [Max Woolf](http://minimaxir.com). See his repo [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple).

# 0. Setup

Run once to install the library

In [1]:
%pip install -q gpt-2-simple

Note: you may need to restart the kernel to use updated packages.


restart the kernel and run the imports

In [2]:
import gpt_2_simple as gpt2
import tensorflow as tf
from datetime import datetime

note the memory usage (0MiB / 32510MiB) for the Tesla V100.
You can re-rerun the above cell to see what memory your code/models are using during training.

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the the current working directory at `/models/<model_name>`.

In [3]:
model_name = "355M" # largest model we can fine-tune

run once to download the file

In [4]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 951Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.72Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 1.54Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:32, 44.0Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 1.16Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.17Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 4.78Mit/s]                                                      


# 1. Generate Text From The Pretrained Model

If you want to generate text from the pretrained model pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`. (This is currently the only way to generate text from the 774M or 1558M models with this notebook.)

In [5]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

2022-05-04 00:23:03.153183: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-04 00:23:04.429042: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading pretrained model models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


## Sample from the model
The follow cell samples from gpt-2, using the provided prefix (seed) and other parameters. It starts the TF session and generates the samples.

Try changing the parameters below to change the output: 
- `prefix` is the prompt. This will be the starting string/seed for your generation. Use your own text. 
- `temperature` sets the variability/randomness of the output. Range 0.0-1.0
- `length` sets the lenght of output (in tokens). max is 1024.


In [6]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="Charizard used",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

Charizard used her Legendary Skill to save her brother, who was knocked unconscious by a blast of electricity.

Pokémon

Pokémon Emerald

Pokémon Emerald is the first game in which the player begins the game as a male. It is the first game to feature the player as a female, though this is later changed to a male in Generation VI.

Trainer Pokémon

Pokémon Emerald's first Pokémon is Jigglypuff, which first appeared in the opening. In the anime, it was
Charizard used a Dragon Pulse to try and knock out her, but it was too late as her Dragon Tail was absorbed by it.

The three of them then watched as a group of Dragon-type Pokémon emerged from the lake. The Pokémon were all in their twenties, each wearing a purple shirt and a black pantsuit. The leader of the group, a young man wearing a blue shirt and black pantsuit, looked like he had a long time to live, but he was already dead.


Charizard used the move Water Gun on Ash's Pikachu. The move's effects, including its ability to hit multi

## Activities
- try varying the prefix. 
  - what length of prefix works best with the given model? 
  - how does the choice of prefix change the format/form of the output.
- try varying the temperature.
- try loading the different sized models (124M, 355M, 774M, 1558M) and generate text without changing the other parameters. 
  - Do you notice any qualitative differences in the output? 

# 2. Fine-Tuning GPT-2

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

In [7]:
#model_name = "355M" # same model as selected above

tf.compat.v1.reset_default_graph()

# check if sess exists (e.g. if we ran section 1 above)
var_exists = 'sess' in locals() or 'sess' in globals()

if not var_exists:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

2022-05-04 00:23:16.363019: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


## Upload a text file
For this, we will use a text file you provide to finetune (continue training) GPT-2. You can use any plain text (.txt) file. 

Simply drag and dropy our text file into the file browser at left. 

Once you have uploaded your file, update the file name in the cell below, then run it.

In [8]:
file_name = "pokemon.txt" # Text file containing the title data

## Run the finetuning

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every `save_every` steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them. If your input text is smaller, training might proceed more quickly.

Other optional-but-helpful parameters for `gpt2.finetune`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [9]:
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=100,
              restore_from='fresh', # change to 'latest' to resume
              run_name='run1',
              print_every=10,
              learning_rate=1e-5,
              sample_every=33,
              save_every=100
              )

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1



2022-05-04 00:23:21.795047: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30979 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.06s/it]


dataset has 314036 tokens
Training...
[10 | 26.26] loss=1.38 avg=1.38
[20 | 46.09] loss=1.11 avg=1.25
[30 | 65.90] loss=1.10 avg=1.19
======== SAMPLE 1 ========
 can't even do the same thing with the same colors? Or are we really limited in terms of color that we're able to give to those that want to use them?

Is there an absolute limit on colors?

So how does color actually work?

It's kind of hard. I've seen it in the movies, and it's very difficult but not impossible. It seems as if each color is only a subset of the whole spectrum. We have our entire spectrum right now (but it's far, far more than that) that we can use to create a color. We call it a spectrum. Each color has a small amount of the same amount of it. In particular, yellow, red, green, blue and purple. Those three colors we used to make a yellow, red and gray color called Tangerine. That color is basically yellow, red and grey, with something in between. When you mix that up some of the energy that goes into that mak

## Notes on finetuning

Keep an eye on the loss, and how quickly it is dropping. A too-rapid drop in loss could be a sign of overfitting, and a learning rate (lr) that is too high. 

After the model is trained, you can download the checkpoint folder to save your work. Training checkpoints are saved to `checkpoint/run1` (or whatever you chose for the run name above).

You can compress it to a rar file and download that. Ask the instructor how.

You're done! Feel free to go to the Generate Text From The Trained Model section to generate text based on your retrained model.

If you have already generated with gpt2, you need to reset the tf graph and gpt2 session. Otherwise, we create a new one:

# 3. Generate Text From The Finetuned Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [10]:
gpt2.generate(sess, run_name='run1') # no prefix, unconditional generation

AP

The Bengals didn't elect to take a shot at the Bengals' defensive tackle, but they did let him go after he failed to make the team.

A source with knowledge of the situation tells PFT that the Bengals have settled on the veteran tackle from the team's practice squad to start at free safety. The move is a way to try to get a production-level player off their hands.

The only player with anything close to a real shot at the start of the season was the injury-prone Owa Odighizuwa, who managed only five tackles in seven games after the practice squad. But the presence of Chris Carter, the second-string tackle at free safety, and Anthony Johnson, who has played in six games in his first year in the league, should help.

Carter, who has been on the practice squad the past three seasons, is the highest-paid veteran on the team.

A fourth-round pick of the Bengals out of Arkansas in 2012, Carter has played in 15 games in his career. He has played 92 percent of Cincinnati's defensive snaps.

In [11]:
gpt2.generate(sess, run_name='run1', prefix="#-------------------------------\n[1]\nName=")

#-------------------------------
[1]
Name=Feraligatr
InternalName=FERALIGATR
Type1=ELECTRIC
Type2=GHOST
BaseStats=100,130,110,105,120,80
GenderRate=Female50Percent
GrowthRate=Medium
BaseEXP=260
EffortPoints=0,0,2,0,0,0
Rareness=90
Happiness=70
Abilities=REGENERATOR,MEGABEATracked
HiddenAbility=DEFIANT
Moves=1,FURYSWIPES,1,CONFUSION,1,REVENGE,1,FAKEOUT,1,MAGNETRISE,1,FAKETEARS,1,LEECHSEED,1,SCARYFACE,1,HYPNOSIS,1,BUBBLEBEAM,1,RAZORLEAF,1,FUTURESIGHT,1,THUNDERWAVE,1,HYPNOSIS,6,SLASH,12,MEGAHORN,16,SAFEGUARD,24,SWEETSCENT,32,STOPPINGWORLD,40,GIGADRAIN,48,BULLETSURF,64,SUNNYDAY,80,SUNNYDAY,100,REVERSAL,120,SAFEGUARD,144,SUPERPOWER
Compatibility=Humanlike
StepsToHatch=5355
Height=2.0
Weight=237.0
Color=Blue
Shape=12
Kind=Gigantic
Pokedex=It is the guardian of a gigantic cave that is believed to hold ancient wisdom. It eats anything that moves in the cave, and even its own body.
BattlerPlayerY=22
BattlerEnemyY=29
BattlerAltitude=13
Evolutions=
#-------------------------------
[2]
Name=Lopunn

## Notes
If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

After testing multiple prefixes and no prefix multiple times, it seemed like a good idea to restrict the outputs a bit to something that is feasible to create. Multiple different games were ran, but "Pokemon, but" seemed to produce very interesting results and fit the YouTube channel's typical videos.

The Temperature of 0.9 seemed to produce the most variation with interesting results without being incoherent.

The length, nsamples, and batch_size were chosen to produce at least 1 full title multiple times with the desired prefix.

In [12]:
gpt2.generate(sess,
              length=25,
              temperature=0.8,
              prefix="#-------------------------------\n[1]\nName=",
              nsamples=5,
              batch_size=5
              )

#-------------------------------
[1]
Name=BarkedGylp
InternalName=BARKEDGYLP
Type1=NORMAL
Base
#-------------------------------
[1]
Name=Windon
InternalName=WINDON
Type1=DARK
Type2=FAKUS
BaseStats=
#-------------------------------
[1]
Name=Porygon2
InternalName=PORYGON2
Type1=NORMAL
BaseStats=45,
#-------------------------------
[1]
Name=Spiral
InternalName=SUNSPIRAL
Type1=NORMAL
BaseStats=80,75
#-------------------------------
[1]
Name=Hawkdon
InternalName=HAWCON
Type1=DARK
BaseStats=80,35,60


For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

The same values were used here as the previous generation, but this time, it produces a txt file and far more results. This provided 100 different titles with the requested prefix.

In [16]:
gen_file = "generated_pokemon6.txt"

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=1000,
                      temperature=0.85,
                      prefix="#-------------------------------\n[1]\nName=",
                      nsamples=100,
                      batch_size=20
                      )

Download the file by hand in the browser at left.

## Load a Trained Model Checkpoint

Uploaded your saved checkpoint and unzip it.

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

This will reset or start the tensorflow session as necessary.

In [ ]:
tf.compat.v1.reset_default_graph()

if not sess:
    sess = gpt2.start_tf_sess()
else:
    sess = gpt2.reset_session(sess)

gpt2.load_gpt2(sess, run_name='run1')

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# License

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Reference
- Max's [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!
- Original repo: [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple) by [Max Woolf](http://minimaxir.com). 
- Original [google colab](https://colab.research.google.com/drive/1VLG8e7YSEwypxU-noRNhsv5dW4NfTGce) from Max.